<span type="title">访问控制权限</span> | <span type="update">2018-07-18</span> - Version <span type="version">1.0</span>
    
    
<span type="intro"><p class="card-text">本章主要介绍Java的访问控制权限。</p></span>

# 权限控制问题

在面向对象语言中，一个较大的问题是类方法的权限问题，如果对一个对象而言，其所有的域和方法都面向使用者开放，这势必造成混乱，更为麻烦的事，对于类的维护者而言，其几乎无法重构一个已经发布的类，因为类的任何域或者方法都有可能被其余程序调用。Python选择了一种“大家都是成年人”的方式，完全放弃了权限控制，这很糟糕。而在高级的Python编程专著中，往往会有一部分章节用来介绍如何将权限控制加回去的。

Java面临的第一个问题是变与不变的问题，对于用户的接口，也就是API，不变，使用 `public` 关键字声明方法或者类，对于内部的接口，使用 `private protected` 来进行其余的分配。这样就可以在不影响接口的情况下大刀阔斧的修改类，进行重构了。

Java面临的第二个问题是，如何区分Array这个类，如果系统实现了它，而用户又实现了一遍？对于Python而言，其默认用户声明的具有较高的优先级（本地目录），然后查找系统自带的库。对于C而言，使用 `include <stdio.h> 或者 include "stdio.h"` 来进行类似的优先级判定。对于Java而言，则是使用 `package 和 import` 进行管理。

# 文件、构件、群组和包

`.java` 文件包含Java源代码，称之为一个编译单元。

对于每一个 `.java` 的文件，当使用 `javac` 工具时，会在同目录生成一个 `.class` 的 JVM 二进制字节码，这两个后缀为 `.java, .class` 的文件合起来称之为一个类的构件。

对于一个构件而言，更改只需要发生在 `.java` 文件即可，当系统调用时，会自动进行 `.class` 文件的更新和处理，不用手动进行。

对于 `.java` 源文件而言，其一般包含一个 `public class ClassName` 的类，这个文件的名称也必须为 `ClassName.java`。并且在这个文件中，不能出现两个 `public class ` 类，虽然可以出现多个 `class` 类，其中这些其余的类具有包访问权限，供主要的类进行调用和处理，在包外不可调用。

当把多个 `.java .class` 类构件放在一个文件夹下，就称之为一个群组。

**包：库单元**

如何告诉Java编译器这些Class文件是一个包呢？对于Python而言，一般需要创建一个空文件夹声明这是一个包，对于Java而言，则必须对每个 `.java` 源文件的**首个非注释行**声明 `package com.mazhangjing.util` 这样一句话，告诉Java这个构件属于 `com.mazhangjing.util` 这个包。也可选择将这个群组打包压缩成 `.jar` 文件，然后将这个文件添加到 `CLASSPATH` 变量中使用。

对于 `com.mzhangjing.util` 包而言，其不是说存在这样一个文件夹叫做 `com.mazhangjing.util`，Java会将句号转换为斜杠，因此这其实是一个三级文件夹，所有的构件都存放在 `com/mazhangjing/util` 这个文件夹下。此外，为了让Java找到这个包，在 `CLASSPATH` 变量中添加 `com` 文件夹存在的父文件夹。

**包和编译的注意事项**

注意，不像Python之类语言会从当前目录开始查找包并自动导入，在 `CLASSPATH` 中如果不存在 `.` 这个目录的话，JAVA是会默认忽略本地文件的。也就是或，当添加 `.` 到变量时，你可以在 `util` 这个目录下执行程序，程序会在 `import util.*` 的时候寻找本地文件夹下的类。

此外，需要注意，对于 `javac` 而言，如果对一个包中的类进行执行的话，如果找不到类，那么添加其包名可能有奇效。`javac com.mazhangjing.util.Print.java` 这样，而不是 `javac Print.java`。因为后者包含 `package com.mazhangjing.util`，在进行编译的时候，它是无法找到当前文件夹下的`/com/mazhangjing/util/Print`这个目录并将自己放进去的。

**导入包中的类**

使用 `import com.mazhangjing.util.Print` 可以导入 `Print` 类，被导入的类的 `com` 文件夹必须在 `CLASSPATH` 能够找到的位置。而使用的这个文件，则可以位于任何地方。

使用 `import com.mazhangjing.util.*` 可以导入 `util` 目录下的所有类，在使用的过程中，需要指明类。

使用 `import static com.mazhangjing.util.Print.*` 可以导入 `Print.class` 类中的所有静态方法，注意，不能用这种写法导入其余方法，一般而言，不能对一个类加`.*`。

```java
/com/mazhangjing/Print.java

package com.mazhangjing;
import java.io.*;
/**
 * Print shortcuts for Study and play Java!
 * @author Corkine Ma
 * @author www.mazhangjing.com
 * @version 0.1
*/
public class Print{
	/**
	 * use for print something by calling System.out.println()
	 * @param obj Object-something
	*/
	public static void print(Object obj){
		System.out.println(obj);
	}
	
	public static void print(){
		System.out.println();
	}
	public static void printnb(Object obj){
		System.out.print(obj);
	}
	public static PrintStream 
	printf(String format, Object... args){
		return System.out.printf(format, args);
	}
	public static void main(String[] args){
		print("Hello");
	}
	
}

import com.mazhangjing.Print;
Print.print("hello");

import com.mazhangjing.*;
Print.print("hello");

import static com.mazhangjing.Print.*
print("hello");
```

# Java 访问权限

如下代码所示：

```java
/com/mazhangjing/Power.java
package com.mazhangjing;
import static com.mazhangjing.Print.*;
//import static Print.*; can't work
public class Power{
	public void pubA(){
		print("Hello from Power class pubA");
	}
	private void priA(){
		print("Hello from Power class priA");
	}
	protected void proA(){
		print("Hello from Power class proA");
	}
}

/somewhere/somename.java
import com.mazhangjing.Power;
class LittlePower extends Power {void proB(){ super.proA();}}
Power a = new Power();
a.pubA();
//a.proA(); //错误
//a.priA(); //错误
LittlePower b = new LittlePower();
b.pubA();
b.proB();
//b.proA(); //错误
```


## 方法的权限

权限一共有四种，从小到大依次是：

- private 权限私有，仅本文件可以访问
- 包访问权限 权限仅限本包内的成员访问
- protected 可以跨越包在别处使用，不过限制只能在继承的子类内部使用，不能直接调用
- public 开放权限，任意位置均可调用

注意上面这个包中的Power类以及其在某个文件中的扩展子类和调用。

很有意思，对于 `public` 而言，其可以在任意地方调用（前提是导入了相关的类，如果不是 static 的话还需要构造这个类对象，下同）。

对于 `private` 而言，其在除了这个类文件的任何地方都不可调用，甚至是包里的其他类也不可以调用。也就是说，其权限锁定在其文件内。

如果不写任何关键字，那么默认这个方法的权限在整个包内，也就是说，属于 `com.mazhangjing` 这个包的所有类文件都可以调用这个方法（再啰嗦一遍，需要import，并且在包内也必须写清楚包名，不能写`import Print`而应该是`import com.mazhangjing.Print`）。

对于`protected`而言，需要说明的是，这是一种专门用来解决跨包，但是又继承了父类的情况，如`LittlePower`而言，被保护的方法可以在其类内部使用，它可以将其包装起来作为自己的公共方法，但是不能使用`proA`,因为这样就变成了继承和重载，不能对于保护方法进行重载，不过换个名字还可以用，所以说，保护方法的权限比公共方法小，比包方法大。

**默认包问题**

如果不声明 `package xxx.xxx.xxx` 的话，Java将你运行的类看作是默认包，这个包可能存在的问题是，你可以对于任意使用 `void methodName(){}` 的方法，也就是具有包权限的方法进行访问，这可能不是你希望的。

```java
/xxx/Cake.java
class Cake {
	public static void main(String[] args){
		Pie x = new Pie();
		x.f();//Pie.f()
    }}

/xxx/Pie.java
class Pie {
	void f() {System.out.println("In Pie.f()");}
}
```

## 类的访问权限

 类只有两种访问权限，`public` 和 包访问权限。如果不希望包的用户访问此类，可以使用 包访问权限，反之，使用公共权限。
 
 ```java
/com/mazhangjing/Copyright.java
package com.mazhangjing;
import static com.mazhangjing.Print.*;
class Copyright{
	public static void getCopyright(){
		print("(c) Marvin Studio 2018");
	}
}
/com/mazhangjing/Power.java
public class Power{
	public Power(){
		print("Init Power...");
	}
	public static void main(String[] args){
		print("Hello from Power class!");
		Copyright.getCopyright(); //可以，在同一包中，具有相同权限
	}
}
/somewhere/somefile.java
Copyright.getCopyright(); // 错误，不可在外部访问
```

# 类重用

## 组合

组合指的是将多个类组合在一起使用，新的类一般包含着其余类的实例，这样就可以在这个类中调用那些类的方法了，这种方法其使用了类的功能，复用了代码而非形式。

```java
import static com.mazhangjing.Print.*;
public class Bird {
	public static void main(String[] args){
		getInfo();
	}
	public static void getInfo(){
		print("Hello from Bird.class");
		Eyes eye = new Eyes();
		print("The bird have a " + eye);
		Foot foot = new Foot();
		print("The bird have " + foot.numOfIt() + " foots.");
		print("The bird's foot is " + foot.colorOfIt() + ".");
	}
}
class Eyes {
	private String info = "blue eyes";
	public String toString() {return info;}
}
class Foot {
	private int num; 
	private String color;
	public int numOfIt(){return num;}
	public String colorOfIt(){
		if (color == null) {color = "red";}
		return color;
	}	
}

Hello from Bird.class
The bird have a blue eyes
The bird have 0 foots.
The bird's foot is red.
```

如上所示，Bird的`getInfo()`方法组合了Eyes和Foot这两个类，在其内部创建了实例（静态的话可以不用）并且调用了其方法。